In [ ]:
import sqlite3
import pandas as pd
import fasttext

In [ ]:
con_usr = sqlite3.connect('user.db')
df_usr = pd.read_sql_query('SELECT * FROM user', con_usr)

In [ ]:
df_usr[df_usr.seo_name.str.match('leon.becker1')]

In [ ]:
con_rev = sqlite3.connect('review.db')
df_rev = pd.read_sql_query("SELECT * from review", con_rev)
df_rev.head()

In [ ]:
df_rev['note_length'] = df_rev.apply(lambda row: len(str(row.note).split()), axis=1)

In [ ]:
df_rev.head()

In [ ]:
df_rev.drop(columns='index', inplace=True)

In [ ]:
df_rev.sort_values(by='id', inplace=True)

In [ ]:
df_rev.to_sql('review', con=con_rev, if_exists='replace')

In [ ]:
PRETRAINED_FASTTEXT_MODEL = 'fasttext_model/lid.176.bin'
fast_model = fasttext.load_model(PRETRAINED_FASTTEXT_MODEL)

In [ ]:
df_rev['len_code'] = df_rev.apply(lambda row: fast_model.predict(str(row.note).replace('\n', ''))[0][0].split('_')[4], axis=1)

In [ ]:
con_rev = sqlite3.connect('review.db')
df_rev = pd.read_sql_query("SELECT * FROM review WHERE len_code = 'en'", con_rev)
df_rev

In [ ]:
df_rev.columns

In [ ]:
con_rev_en = sqlite3.connect('review_en.db')
cur_rev_en = con_rev_en.cursor()
cur_rev_en.execute('CREATE TABLE IF NOT EXISTS review_en('
                   'id, wine_id, rating, note, created_at, likes_count, comments_count, scan_image_path, user_id, note_length, len_code)')

In [ ]:
df_rev.to_sql('review_en', con=con_rev_en, if_exists='replace')